In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import utils as np_utils
from keras.utils import to_categorical
print('Using TensorFlow version', tf.__version__)

Using TensorFlow version 2.3.0


In [2]:
import pandas as pd 
import io

url_train = 'https://raw.githubusercontent.com/bondadakumarsai/DeeplearningbasedIQClassification/master/IQ_samples/trainDataLabels.csv'
url_test = 'https://raw.githubusercontent.com/bondadakumarsai/DeeplearningbasedIQClassification/master/IQ_samples/testData.csv'

In [3]:
df_train = pd.read_csv(url_train)
df_test  = pd.read_csv(url_test)

In [4]:
df_train['Input'] = df_train['Input'].str.replace('i','j').apply(lambda x: np.complex(x))
data = df_train.values
X_train = data[:,1]
X_train = X_train.reshape(len(X_train),1)
Y_train = data[:,2].real.reshape(len(X_train),1)
#print(X_train[0:5])
#print(Y_train[0:5])
#print(X_train.shape)
#print(Y_train.shape)



In [5]:
df_test['Input'] = df_test['Input'].str.replace('i','j').apply(lambda x: np.complex(x))
data = df_test.values
X_test = data[:,1]
X_test = X_test.reshape(len(X_test),1)
print(X_test.shape)

(10000, 1)


In [6]:
y_train_encoded = to_categorical(Y_train)
print('y_train shape: ', y_train_encoded.shape)


y_train shape:  (20000, 4)


In [7]:
y_train_encoded[:2]

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [8]:

#x_train_reshaped = np.reshape(X_train, (20000, 1))
x = abs(X_train)
x_phase = np.angle(X_train)
x_input = np.concatenate((x, x_phase, x**2, x_phase**2 ,x**4, x_phase**4, x**6, x_phase**6, x**8, x_phase**8),axis=1)

x = abs(X_test)
x_phase = np.angle(X_test)
x_input_test = np.concatenate((x, x_phase, x**2, x_phase**2 ,x**4, x_phase**4, x**6, x_phase**6, x**8, x_phase**8),axis=1)

print('x: ', x_input)

x:  [[ 1.02095667e+00  2.44247384e+00  1.04235252e+00 ...  2.12314435e+02
   1.18047961e+00  1.26659965e+03]
 [ 1.08718317e+00  1.90564799e+00  1.18196724e+00 ...  4.78912407e+01
   1.95173907e+00  1.73916766e+02]
 [ 5.02102051e-01  7.04897719e-01  2.52106469e-01 ...  1.22675159e-01
   4.03957765e-03  6.09549307e-02]
 ...
 [ 1.36096803e+00  2.36468943e+00  1.85223398e+00 ...  1.74841553e+02
   1.17701877e+01  9.77671316e+02]
 [ 9.96447061e-01 -2.33123428e+00  9.92906746e-01 ...  1.60514963e+02
   9.71927443e-01  8.72343170e+02]
 [ 8.10456060e-01  2.77833792e+00  6.56839026e-01 ...  4.59949758e+02
   1.86138321e-01  3.55042650e+03]]


In [9]:
print(x_input)

[[ 1.02095667e+00  2.44247384e+00  1.04235252e+00 ...  2.12314435e+02
   1.18047961e+00  1.26659965e+03]
 [ 1.08718317e+00  1.90564799e+00  1.18196724e+00 ...  4.78912407e+01
   1.95173907e+00  1.73916766e+02]
 [ 5.02102051e-01  7.04897719e-01  2.52106469e-01 ...  1.22675159e-01
   4.03957765e-03  6.09549307e-02]
 ...
 [ 1.36096803e+00  2.36468943e+00  1.85223398e+00 ...  1.74841553e+02
   1.17701877e+01  9.77671316e+02]
 [ 9.96447061e-01 -2.33123428e+00  9.92906746e-01 ...  1.60514963e+02
   9.71927443e-01  8.72343170e+02]
 [ 8.10456060e-01  2.77833792e+00  6.56839026e-01 ...  4.59949758e+02
   1.86138321e-01  3.55042650e+03]]


In [10]:
x_mean = np.mean(x_input,axis=0)#reshape(x_input.shape[1],1)
x_mean = x_mean.reshape(1,x_input.shape[1])
x_std = np.std(x_input,axis =0)
x_std = x_std.reshape(1, x_input.shape[1])

In [11]:
x_mean_test = np.mean(x_input_test,axis=0)#reshape(x_input.shape[1],1)
x_mean_test = x_mean.reshape(1,x_input_test.shape[1])
x_std_test = np.std(x_input,axis =0)
x_std_test = x_std.reshape(1, x_input_test.shape[1])

In [12]:
print('mean: ', x_mean.shape)
print('std: ', x_std.shape)

mean:  (1, 10)
std:  (1, 10)


In [13]:
epsilon = 1e-10
x_train_norm = (x_input - x_mean)/(x_std + epsilon)
x_test_norm = (x_input_test - x_mean_test)/(x_std_test + epsilon)

In [14]:
print(set(x_train_norm[0]))

{0.19500166658276846, 1.3592900815397813, -0.16398144827080052, 1.0351476532964838, 0.8822529935513699, -0.27344868811842354, 0.6418932990223964, -0.3028816310034469, 0.40639998470983435, 0.059812939300189195}


In [15]:
from tensorflow.keras.initializers import GlorotNormal
initializer = tf.keras.initializers.GlorotNormal(seed=1)

from tensorflow.keras.optimizers.schedules import ExponentialDecay

#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
#optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

model = Sequential([
    Dense(512, activation = 'relu', input_shape = (10,),kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(4, activation = 'softmax',kernel_initializer=initializer)
])

In [17]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               5632      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               2

In [18]:
h = model.fit(
    x_train_norm,
    y_train_encoded,
    batch_size = 1024,
    epochs = 50
)

Epoch 1/50
20/20 [==============================] - 5s 248ms/step - loss: 1.0196 - accuracy: 0.4300
Epoch 2/50
20/20 [==============================] - 5s 248ms/step - loss: 0.7248 - accuracy: 0.6061
Epoch 3/50
20/20 [==============================] - 5s 249ms/step - loss: 0.6523 - accuracy: 0.6179
Epoch 4/50
20/20 [==============================] - 5s 248ms/step - loss: 0.5348 - accuracy: 0.7168
Epoch 5/50
20/20 [==============================] - 5s 248ms/step - loss: 0.4699 - accuracy: 0.7890
Epoch 6/50
20/20 [==============================] - 5s 248ms/step - loss: 0.4327 - accuracy: 0.8198
Epoch 7/50
20/20 [==============================] - 5s 246ms/step - loss: 0.3915 - accuracy: 0.8383
Epoch 8/50
20/20 [==============================] - 5s 247ms/step - loss: 0.3692 - accuracy: 0.8497
Epoch 9/50
20/20 [==============================] - 5s 245ms/step - loss: 0.3672 - accuracy: 0.8504
Epoch 10/50
20/20 [==============================] - 5s 250ms/step - loss: 0.3696 - accuracy: 0.8464

In [19]:
loss, accuracy = model.evaluate(x_test_norm)

print('test set accuracy: ', accuracy * 100)

313/313 [==============================] - 0s 669us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
test set accuracy:  0.0


In [20]:
preds = model.predict(x_test_norm)
pred = np.argmax(preds,axis = 1).reshape(10000,1)
id = np.arange(1,10001).reshape(10000,1)
prediction = np.concatenate((id, pred),axis=1)

import csv   
      
# field names   
fields = ['Id', 'Predicted Label']  

filename = "submission.csv"
      
# writing to csv file   
with open(filename, 'w') as csvfile:   
    # creating a csv writer object   
    csvwriter = csv.writer(csvfile)   
          
    # writing the fields   
    csvwriter.writerow(fields)   
          
    # writing the data rows   
    csvwriter.writerows(prediction) 